In [4]:
import pandas as pd, numpy as np

# CSV: ema half-life = 50 steps
df = pd.read_csv("450m-8xg34b-val2.86.csv",  engine="python")  # columns: tokens step loss ema
res = df["loss"] - df["ema"]
ema_hl = 50
res_var = res.pow(2).ewm(halflife=ema_hl, adjust=False).mean()
z = res / np.sqrt(res_var + 1e-12)
p3 = (z.abs() > 3).mean()
r1 = np.corrcoef(res[1:], res[:-1])[0,1]
dnr = res.pow(2).mean()**0.5 / df["ema"].diff().abs().replace(0,np.nan).ewm(halflife=ema_hl, adjust=False).mean().mean()
print({"p_abs_z_gt_3": float(p3), "rho_lag1": float(r1), "DNR": float(dnr)})


{'p_abs_z_gt_3': 0.00550314465408805, 'rho_lag1': -0.25351749354912473, 'DNR': 1.4357276529788743}


- p(|z|>3) = 0.0055 (0.55%): low tail rate → volatility is well-behaved
- ρ₁ = −0.254: mild negative autocorrelation → slight overshoot/undershoot around the EMA
- DNR = 1.44: drift and noise are comparable; typical for a stable regime